In [3]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import re

from matplotlib.axes import Axes
from pandas import DataFrame 
from numpy import ndarray

In [4]:
filenames = [
    "round2-large-non_row-13291769969404998015",
    "round2-large-row-13291769969404998015",
    "round2-narrow-non_row-13291769969404998015",
    "round2-narrow-row-13291769969404998015"
]

In [5]:

def extract_min_dist(s):
    match = re.search(r'"min_dist"\s*=>\s*([\d\.]+)', s)
    if match:
        return float(match.group(1))
    return None

def get_results(fn: str) -> DataFrame:
    parts = fn.split("-")
    rac_type, maneuver = parts[1:3]
    
    df = pd.read_csv(f"../results/round2/{fn}.csv")    
    df["severity"] = (2000.0 - df.min_dist) / 2000.0
    df.loc[df.min_dist > 2000.0, "severity"] = 0.0
    
    return df[["contactLevel", "scenario", "drone_response_distance", "min_dist", "severity"]]

def reorder(df: DataFrame, head: list[str]) -> DataFrame:
    return df[[*head, *[c for c in df.columns if c not in head]]]

In [6]:
results = {fn : get_results(fn) for fn in filenames}

In [26]:
def stats(
    df: DataFrame,
) -> DataFrame:
    """
    Returns a dataframe of how each scenario performed in terms of risk.
    @df : the dataframe containing the results from the simulation.
    """
    
    df = df.groupby("scenario").agg(
        simcount = ("contactLevel", "count"),
        n_viol = ("contactLevel", lambda x: (x != "none").sum()),
        sevsum = ("severity", "sum")
    )
    df["mean_severity"] = df.sevsum / df.n_viol 
    df["probability"] = df.n_viol / df.simcount
    df["risk"] = df.probability * df.mean_severity
    
    return df[df.index != "noContact"]

def risk_over_distance(results: dict[str, DataFrame]) -> DataFrame:
    # Returns a dataframe containing risk for each response distance.
    df = None
    
    for title, data in results.items():
        parts = title.split("-")
        
        rac_type, maneuver = parts[1:3]
        

        data = data.groupby(["scenario", "drone_response_distance"], as_index=False).agg(
            sim_count=("severity", "size"),
            n_viol=("contactLevel", lambda x: (x != "none").sum()),
            sevsum=("severity", "sum"),
        ).reset_index()
        
        data["title"] = title
        data["mean_severity"] = data.sevsum / data.n_viol 
        data["probability"] = data.n_viol / data.sim_count
        data["risk"] = data.probability * data.mean_severity

        data = data.pivot(index="scenario", columns="drone_response_distance", values="risk")
        data["rac_type"] = rac_type
        data["maneuver"] = maneuver
        data = reorder(data, ["rac_type", "maneuver"])
        
        if df is not None:
            df = pd.concat([df, data])
        else:
            df = data
    
    
    return df[df.index != "noContact"]

In [24]:
risk_over_distance(results).to_csv("../results/round2/analysis/riskXdist-large.csv")

In [27]:
[
    (title, stats(df)) for title, df in results.items()   
]

for title, df in results.items():
    stats(df).to_csv(f"../results/round2/analysis/stats-{title}.csv")

# df = risk_over_distance(results)

In [16]:
df

drone_response_distance,scenario,maneuver,3000.0,3700.0,4400.0,5100.0,5800.0,6500.0,7200.0,7900.0,8600.0,9300.0,10000.0
0,round2-large-non_row,13291769969404998015,0.091137,0.043418,0.024319,0.010330,0.005762,0.002421,0.001426,0.000594,0.000866,0.002062,0.003924
1,round2-large-row,13291769969404998015,0.317819,0.315038,0.312759,0.300517,0.290666,0.252782,0.231767,0.217642,0.207225,0.194729,0.190306
2,round2-narrow-non_row,13291769969404998015,0.128854,0.070376,0.043556,0.020840,0.010551,0.005100,0.003209,0.001998,0.002311,0.003735,0.009551
3,round2-narrow-row,13291769969404998015,0.321699,0.292155,0.276681,0.258220,0.238659,0.216428,0.204185,0.180693,0.148840,0.129506,0.118703


In [69]:
df.to_csv("tmp.csv")
df

PermissionError: [Errno 13] Permission denied: 'tmp.csv'

In [67]:
compare(results, "converging-adverse")

KeyError: 'cs528-converging-adverse-horizontal-params'